In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
project_path = os.getcwd() + '/../'
sys.path.append(project_path + "utils")

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
import sklearn

Populating the interactive namespace from numpy and matplotlib


In [2]:
import seaborn as sns

/Users/admin/Desktop/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
import function
import preprocessing
import local_bayes
import visualize

In [4]:
sys.path.append(project_path + "models/cnn")

In [19]:
p = preprocessing.Preprocessing(data_dir = "../original_data/bookmark/")

# data load

In [28]:
category_ts = [[1, 2, 3], [2, 3, 4], [3, 4, 5], [4, 5, 6], [5, 6, 7]]
train_test_ratio = 0.8

In [29]:
category_ts_dict = p.div_train_test(category_ts, train_test_ratio)

In [30]:
category_ts_dict['train']

array([[5, 6, 7],
       [2, 3, 4],
       [1, 2, 3],
       [4, 5, 6]])

In [31]:
category_ts_dict['test']

array([[3, 4, 5]])

In [32]:
## 注意: このデータロードはdataを入力とする場合, 関数ごとに毎回必要
# movielens = p.load_data('movielens.twd')
# science = p.load_data('science.twd')
# ajax = p.load_data('ajax.twd')
# java = p.load_data('java.twd')
# news = p.load_data('news.twd')
# css = p.load_data('css.twd')
# javascript = p.load_data('javascript.twd')
# opensource = p.load_data('opensource.twd')
# webdesign = p.load_data('webdesign.twd')
# design = p.load_data('design.twd')
# linux = p.load_data('linux.twd')
# photography = p.load_data('photography.twd')

# data_ls = [science, ajax, java, news, css, javascript, opensource, webdesign, design, linux, photography]
data_name_ls = ['science', 'ajax', 'java', 'news', 'css', 'javascript', 'opensource', 'webdesign', 'design', 'linux', 'photography']

In [35]:
train_test_ratio = 0.8
balance = False
for data_name in data_name_ls:
    print data_name
    data = p.load_data('%s.twd' % data_name)
    category_ts, category_sum = p.timeseries(data)
    category_ts_dict = p.div_train_test(category_ts, train_test_ratio)
    category_started_ts_dict = p.make_started_ts(category_ts_dict)
    category_input_ts_dict, category_label_dict, category_input_sum_dict = p.make_supervised_data(category_started_ts_dict, stride=5)
    if balance:
        category_input_ts_dict, category_label_dict = p.balancing_sampling(category_input_ts_dict, category_label_dict)
    save_data_dir = '../data/bookmark/%s/' % data_name
    os.mkdir(save_data_dir)
    p.make_test_dataset(category_input_ts_dict, category_label_dict, save_data_dir)

science
186 data created

OSError: [Errno 17] File exists: '../data/bookmark/science/'

In [45]:
for data_name in data_name_ls:
    print data_name
    data = p.load_data('%s.twd' % data_name)
    category_ts, category_sum = p.timeseries(data)
    category_started_ts = p.make_started_ts(category_ts)
    category_input_ts, category_label, category_input_sum = p.make_supervised_data(category_started_ts, stride=30)
    x, label = p.balancing_sampling(category_input_ts, category_label)
    save_data_dir = '../data/bookmark/%s_30/' % data_name
    os.mkdir(save_data_dir)
    p.make_test_dataset(x, label, save_data_dir)

science
336 data createdajax
2710 data createdjava
410 data creatednews
1157 data createdcss
4084 data createdjavascript
2386 data createdopensource
1258 data createdwebdesign
4008 data createddesign
5503 data createdlinux
1661 data createdphotography
1586 data created

In [46]:
for data_name in data_name_ls:
    print data_name
    data = p.load_data('%s.twd' % data_name)
    category_ts, category_sum = p.timeseries(data)
    category_started_ts = p.make_started_ts(category_ts)
    category_input_ts, category_label, category_input_sum = p.make_supervised_data(category_started_ts, stride=60)
    x, label = p.balancing_sampling(category_input_ts, category_label)
    save_data_dir = '../data/bookmark/%s_60/' % data_name
    os.mkdir(save_data_dir)
    p.make_test_dataset(x, label, save_data_dir)

science
247 data createdajax
1819 data createdjava
280 data creatednews
622 data createdcss
2583 data createdjavascript
1686 data createdopensource
747 data createdwebdesign
2635 data createddesign
3539 data createdlinux
1212 data createdphotography
1050 data created

In [48]:
# all_save_dir = '../original_data/bookmark/'
# all_category_ts = []
# for data_name in data_name_ls:
#     print data_name
#     data = p.load_data('%s.twd' % data_name)
#     category_ts, category_sum = p.timeseries(data, sum_min=100)
#     all_category_ts.extend(category_ts)
    
# numpy.savez(all_save_dir + 'all_category.npz', numpy.array(all_category_ts))

# category_started_ts = p.make_started_ts(all_category_ts)
# category_input_ts, category_label, category_input_sum = p.make_supervised_data(category_started_ts, stride=30)
# x, label = p.balancing_sampling(category_input_ts, category_label)
# save_data_dir = '../data/bookmark/all/'
# os.mkdir(save_data_dir)
# p.make_test_dataset(x, label, save_data_dir, rand=True)    

In [49]:
save_data_dir = '../data/bookmark/ajax/'
train_0_0 = numpy.load(save_data_dir + '0_test/0_fold/0_train.npz')
test_0_0 = numpy.load(save_data_dir + '0_test/0_fold/0_test.npz')

In [50]:
train_0_0['x'].shape

(9513, 30)

In [51]:
test_0_0['y'][:10]

array([1, 0, 1, 1, 0, 0, 0, 1, 1, 1])

# model training

In [52]:
import cnn

In [53]:
epoch_num = 50
save_data_dir = '../data/bookmark/ajax/'
save_model_dir = "../models/cnn/results/ajax_%s/" % epoch_num
os.mkdir("../models/cnn/results/ajax_%s" % epoch_num)
train_0_0 = numpy.load(save_data_dir + '0_test/0_fold/0_train.npz')
test_0_0 = numpy.load(save_data_dir + '0_test/0_fold/0_test.npz')
cnntrain = cnn.CNN(save_model_dir=save_model_dir, epoch_num=epoch_num)

In [54]:
train_0_0['x'].shape

(9513, 30)

In [55]:
cnntrain.train(train_0_0, test_0_0)

epoch 1
graph generated
train mean loss=0.763555543419, accuracy=0.667087147022, throughput=985.019075647 images/sec
test  mean loss=0.655925751986, accuracy=0.701850297674
epoch 2
train mean loss=0.587689072254, accuracy=0.700620207271, throughput=1061.56957025 images/sec
test  mean loss=0.66575453718, accuracy=0.672834321468
epoch 3
train mean loss=0.559981701979, accuracy=0.711237257147, throughput=1055.22101949 images/sec
test  mean loss=0.531552276534, accuracy=0.718250631183
epoch 4
train mean loss=0.546330547378, accuracy=0.719016085184, throughput=1067.50973784 images/sec
test  mean loss=0.538945324086, accuracy=0.723717412094
epoch 5
train mean loss=0.523404573854, accuracy=0.730789448596, throughput=1067.28881073 images/sec
test  mean loss=0.531542739766, accuracy=0.735912538507
epoch 6
train mean loss=0.517464605937, accuracy=0.737201724521, throughput=1072.68237419 images/sec
test  mean loss=0.516535560081, accuracy=0.732127839422
epoch 7
train mean loss=0.495763141519, acc

In [56]:
epoch_num = 50
save_data_dir = '../data/bookmark/ajax_30/'
save_model_dir = "../models/cnn/results/ajax_%s_30/" % epoch_num
os.mkdir("../models/cnn/results/ajax_%s_30" % epoch_num)
train_0_0_30 = numpy.load(save_data_dir + '0_test/0_fold/0_train.npz')
test_0_0_30 = numpy.load(save_data_dir + '0_test/0_fold/0_test.npz')
cnntrain = cnn.CNN(save_model_dir=save_model_dir, epoch_num=epoch_num)

In [57]:
train_0_0_30['x'].shape

(2168, 30)

In [58]:
cnntrain.train(train_0_0_30, test_0_0_30)

epoch 1
graph generated
train mean loss=0.856766545025, accuracy=0.749077491297, throughput=915.436561686 images/sec
test  mean loss=0.770515868255, accuracy=0.769372691967
epoch 2
train mean loss=0.513243945982, accuracy=0.783671582042, throughput=951.976112163 images/sec
test  mean loss=0.517006267701, accuracy=0.7822878176
epoch 3
train mean loss=0.412609556917, accuracy=0.818265679633, throughput=1027.25791076 images/sec
test  mean loss=0.451953402415, accuracy=0.789667895359
epoch 4
train mean loss=0.374739015839, accuracy=0.839022137032, throughput=1065.5489713 images/sec
test  mean loss=0.436411118925, accuracy=0.813653138291
epoch 5
train mean loss=0.350621334466, accuracy=0.845479698474, throughput=1073.68401146 images/sec
test  mean loss=0.52345504488, accuracy=0.789667890521
epoch 6
train mean loss=0.344226040468, accuracy=0.855166042038, throughput=1046.9965709 images/sec
test  mean loss=0.429894789449, accuracy=0.800738004301
epoch 7
train mean loss=0.33875343829, accuracy

In [59]:
epoch_num = 50
save_data_dir = '../data/bookmark/ajax_60/'
save_model_dir = "../models/cnn/results/ajax_%s_60/" % epoch_num
os.mkdir("../models/cnn/results/ajax_%s_60" % epoch_num)
train_0_0_60 = numpy.load(save_data_dir + '0_test/0_fold/0_train.npz')
test_0_0_60 = numpy.load(save_data_dir + '0_test/0_fold/0_test.npz')
cnntrain = cnn.CNN(save_model_dir=save_model_dir, epoch_num=epoch_num)

In [60]:
train_0_0_60['x'].shape

(1456, 30)

In [61]:
cnntrain.train(train_0_0_60, test_0_0_60)

epoch 1
graph generated
train mean loss=0.756237555234, accuracy=0.770604389505, throughput=414.482740029 images/sec
test  mean loss=0.714041425916, accuracy=0.834710748728
epoch 2
train mean loss=0.495513381518, accuracy=0.840659337139, throughput=712.392200358 images/sec
test  mean loss=0.427742324174, accuracy=0.867768587324
epoch 3
train mean loss=0.323671543909, accuracy=0.870879124563, throughput=818.748929825 images/sec
test  mean loss=0.292671098484, accuracy=0.895316805721
epoch 4
train mean loss=0.295001422024, accuracy=0.886675816889, throughput=917.035975479 images/sec
test  mean loss=0.380116275981, accuracy=0.887052334373
epoch 5
train mean loss=0.34921425846, accuracy=0.860576918901, throughput=952.15286567 images/sec
test  mean loss=0.473161475687, accuracy=0.878787879445
epoch 6
train mean loss=0.262986373893, accuracy=0.901098895122, throughput=911.876287539 images/sec
test  mean loss=0.245746021572, accuracy=0.925619831755
epoch 7
train mean loss=0.208810764692, accu